In [15]:
import os

In [16]:
%pwd

'c:\\Users\\chana\\OneDrive\\Desktop\\MLStudy\\MLFlow'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\chana\\OneDrive\\Desktop\\MLStudy\\MLFlow'

In [17]:
## Need to provide this to upload mlruns to ml flow; Else it will be saved locally.

# os.environ["MLFLOW_TRACKING_URI"]="XXX"
# os.environ["MLFLOW_TRACKING_USERNAME"]="XXX"
# os.environ["MLFLOW_TRACKING_PASSWORD"]="XXX"    

In [18]:
# # Run the following in anaconda to set environment variables.
#conda env config vars set MLFLOW_TRACKING_URI=XXX
#conda env config vars set MLFLOW_TRACKING_USERNAME=XXX
#conda env config vars set MLFLOW_TRACKING_PASSWORD=XXX


In [19]:
import tensorflow as tf

In [20]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [21]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [22]:
from src.cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [23]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        config = self.config.evaluation

        eval_config = EvaluationConfig(
            path_of_model=config.trained_model_path,
            training_data=config.training_data,
            mlflow_uri=config.mlflow_uri,
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [24]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [25]:
os.environ

environ{'ALLUSERSPROFILE': 'C:\\ProgramData',
        'APPDATA': 'C:\\Users\\chana\\AppData\\Roaming',
        'CHROME_CRASHPAD_PIPE_NAME': '\\\\.\\pipe\\crashpad_22960_QFXIVIIHMXZMWJWT',
        'COMMONPROGRAMFILES': 'C:\\Program Files\\Common Files',
        'COMMONPROGRAMFILES(X86)': 'C:\\Program Files (x86)\\Common Files',
        'COMMONPROGRAMW6432': 'C:\\Program Files\\Common Files',
        'COMPUTERNAME': 'LAPTOP-CTJP9GEC',
        'COMSPEC': 'C:\\WINDOWS\\system32\\cmd.exe',
        'CONDA_DEFAULT_ENV': 'cancer',
        'CONDA_EXE': 'C:\\Users\\chana\\anaconda3\\Scripts\\conda.exe',
        'CONDA_PREFIX': 'C:\\Users\\chana\\anaconda3\\envs\\cancer',
        'CONDA_PROMPT_MODIFIER': '(cancer) ',
        'CONDA_PYTHON_EXE': 'C:\\Users\\chana\\anaconda3\\python.exe',
        'CONDA_ROOT': 'C:\\Users\\chana\\anaconda3',
        'CONDA_SHLVL': '1',
        'CUDA_PATH': 'C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v11.8',
        'CUDA_PATH_V11_8': 'C:\\Program Files\\

In [26]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [27]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-02-25 00:00:35,817: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-25 00:00:35,819: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-25 00:00:35,820: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 6s 831ms/step - loss: 1.9054 - accuracy: 0.7549
[2024-02-25 00:00:42,303: INFO: common: json file saved at: scores.json]
[2024-02-25 00:00:42,331: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)': /Chanakarn-Luangpirom/MLFlow.mlflow/api/2.0/mlflow/runs/create]


2024/02/25 00:00:44 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-02-25 00:00:44,983: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\chana\AppData\Local\Temp\tmpaz6a5cez\model\data\model\assets
[2024-02-25 00:00:45,466: INFO: builder_impl: Assets written to: C:\Users\chana\AppData\Local\Temp\tmpaz6a5cez\model\data\model\assets]


c:\Users\chana\anaconda3\envs\cancer\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'VGG16Model'.
2024/02/25 00:01:26 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
